# 2.1 Ingesta de Datos Estructurados
----------------

Esta semana, estudiaremos el API de Spark dedicado a la lectura, escritura y procesamiento de Dataframes

Agenda

1. Creación de Dataframes
2. Lectura de Dataframes
3. Exploración de Datos
4. Escritura de Dataframes


  <img src= "http://1.bp.blogspot.com/-BklNJ2CjC3s/VejWK5i2WHI/AAAAAAAAAVU/ih103fAbUGY/s640/Screen%2BShot%2B2015-09-03%2Bat%2B4.21.21%2BPM.png">
  
  
Para ello cargaremos un `DataFrame` y reconoceremos algunos datos de este

<img src='https://pandas.pydata.org/docs/_images/02_io_readwrite.svg'>

In [5]:
!python -V

Python 3.7.11


In [2]:
# GOOGLE COLAB -> inicializamos spark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## 1. Creación Dataframe


Es posible crear un dataframe de Spark a partir de información de una lista de python

<h2>1.1 A partir de Filas de Datos</h2>

In [3]:
from datetime import datetime, date
from pyspark.sql import Row


df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



<h2>1.2 A partir de una Lista de Tuplas con un Esquema</h2>

In [ ]:
data = [
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
]

dll_schema = 'a long, b double, c string, d date, e timestamp' 

df = spark.createDataFrame(data, schema=dll_schema)
df.show(20)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



<h2>1.3 A partir de un Dataframe de Pandas</h2>

In [ ]:
# Documentacion Pandas : https://pandas.pydata.org/docs/getting_started/index.html#getting-started
import pandas as pd

dicx = {
  'nombres': ['gonzalo', 'luis'],
  'apellidos': ['Delgado', 'Picasso']
}

# pandas
df_pandas = pd.DataFrame(dicx)
df_pandas.head()

,nombres,apellidos
0,gonzalo,Delgado
1,luis,Picasso


In [ ]:
# pandas to Spark
df_spark = spark.createDataFrame(df_pandas)
df_spark.show(5)

+-------+---------+
|nombres|apellidos|
+-------+---------+
|gonzalo|  Delgado|
|   luis|  Picasso|
+-------+---------+



## 2. Lectura de Datos

### Lectura de Archivos CSV

In [ ]:
flightCsvPath = "/FileStore/2015_summary.csv"

flightDF = (spark.read
  .option("sep", ",") #opcion separador de archivo
  .option("header", True) # contiene cabecera?
  .option("inferSchema", True) # inferir tipo de datos de columnas
  .csv(flightCsvPath))

flightDF.show(2, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
+-----------------+-------------------+-----+
only showing top 2 rows



Es factible definir el esquema de forma manual creando `StructType` con los nombres de columna y tipo de datos


Listado de tipos de datos, [link](https://spark.apache.org/docs/latest/sql-ref-datatypes.html)

<h4> Definición del esquema para PySpark con StructType </h4>

In [ ]:
from pyspark.sql.types import LongType, StringType, StructType, StructField
# 3° parametro es para si acepta o no NULL
userDefinedSchema = StructType([
  StructField("user_id", StringType(), True),
  StructField("user_first_touch_timestamp", LongType(), True),
  StructField("email", StringType(), True)
])

In [ ]:
usersDF = (spark.read
  .option("sep", "\t")
  .option("header", True)
  .schema(userDefinedSchema)
  .csv(usersCsvPath))

<h4> Definición del esquema como DDL </h4>

In [ ]:
DDLSchema = "user_id string, user_first_touch_timestamp long, email string"

# tambien podemos colocar las opciones en un diccionario y usar 'options' en lugar de 'option'
options = {
  'sep': '\t',
  'header': True
}

usersDF = (spark.read
  .options(**options)
  .schema(DDLSchema)
  .csv(usersCsvPath))

## 3. Exploración DataFrame

La exploración de datos es la etapa donde obtenemos informacion sobre la data cargada, información como: cantidad de registros, visualizar registros de la data, cantidad de columnas, tipos de datos de las columnas, nombre de columnas, etc

`show` nos devuelve un vistazo a los primeros 20 registros del Dataframe

In [ ]:
flightDF.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

<h3>3.1 Parametros del Metodo Show<h3>

In [ ]:
# Podemos añadirle parametros para mostrar la información de diferente manera

df.show(n=2, # cantidad registros
        truncate=False,  # evitar el truncamiento de los registros
        vertical=True  # mostrar data en forma vertical
       )

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
-RECORD 1------------------
 a   | 2                   
 b   | 3.0                 
 c   | string2             
 d   | 2000-02-01          
 e   | 2000-01-02 12:00:00 
only showing top 2 rows



<h3>3.2 Limit<h3>

In [ ]:
# limit -> Toma las 'n' primeras filas del df | funciona como top en sql

df.limit(10)

Out[3]: DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

<h3>3.3 Conf<h3>

In [ ]:
# nos permite obtener una mejor visualización del DF

spark.conf('spark.sql.repl.eagerEval.enabled', True) # Configuraciones van al inicio

df.limit(10)

<h3>3.4 Display<h3>

In [ ]:
# SOLO DATABRICKS
# Nos muestra los datos en forma de tabla con las opciones de graficados dinamicos como los harias en excel
display(df)


<h3>3.5 printSchema<h3>

In [ ]:
# printSchema retorna los nombres de columna y su tipo de datos
flightDF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



<h3>3.6 columns<h3>

In [ ]:
# columns retorna unicamente el nombre de columna
flightDF.columns

Out[10]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

<h3>3.7 dtypes<h3>

In [ ]:
# dtypes retorna el nombre de columna y tipo de dato en forma de lista de tupla
flightDF.dtypes

Out[11]: [('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'int')]

<h3>3.7 describe<h3>

In [ ]:
# describe: retorna una descripción de los datos de las columnas del DataFrame
flightDF.describe().show()

+-------+-----------------+-------------------+------------------+
|summary|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|             count|
+-------+-----------------+-------------------+------------------+
|  count|              256|                256|               256|
|   mean|             null|               null|       1770.765625|
| stddev|             null|               null|23126.516918551915|
|    min|          Algeria|             Angola|                 1|
|    max|           Zambia|            Vietnam|            370002|
+-------+-----------------+-------------------+------------------+



## 3. Escritura

Spark ofrece la **escritura de datos** de la siguiente manera:

<code>DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(
  ...).save() </code>
  
Donde:
  - format: formato a escribir el archivo
  - option: opciones para la escritura (ejemplo: en CSV guardar con este tipo de separador, en parquet este tipo de comprensión)
  - partitionBy:  particion del archivo
  - save: ruta a guardar el archivo
  
Tambien se establecen los siguiente modos de escritura:

| Write Mode     	| Description                                                                                       	|
|---------------	|---------------------------------------------------------------------------------------------------	|
| append        	| Agrega los archivos de salida a la lista de archivos  que ya existen en esa ubicación             	|
| overwrite     	| Sobrescribirá por completo cualquier dato que ya  exista en la ruta especificada                  	|
| errorIfExists 	|  Lanza un error y falla la escritura si ya existen datos  o archivos en la ubicación especificada 	|

#### 3.1 Escribiendo archivo CSV

In [ ]:
path_out = 'path/nameFolder'

(df.write.format("csv")
   .mode("overwrite")
   .option("sep", "\t")
   .save(path_out)
)

#### 3.2 Escribiendo archivo Avro

In [ ]:
# El formato de archivo AVRO está asociado al sistema de serialización de datos de Apache Hadoop llamado Apache Avro
path = "Folder1/Data_Avro"

df.write.format("avro").save(path)

#### 3.3 Escribiendo archivo Parquet

In [ ]:
# Parquet es el formato de almacenamiento en columnas principal en el ecosistema Hadoop.
usersOutputPath = ''

(df.write
  .option("compression", "snappy")
  .mode("overwrite")
  .parquet(usersOutputPath)
)

# EJERCICIOS
------------------

### 1. Lectura con inferencia de Esquema
- Cargue el archivo <code>.csv</code> 
- Cree `productsDF` leyendo el archivo CSV recien cargado
- Configure las opciones para tomar la primera linea como cafecera e infiera el esquema
- Muestre algunos datos básicos del Df

In [3]:
import os
import shutil

# solo google colab
import io
from google.colab import files

In [4]:
# Cargando data Google Colab
uploaded = files.upload()

Saving 201508_station_data.csv to 201508_station_data.csv


In [5]:
os.listdir()

['.config', '201508_station_data.csv', 'sample_data']

In [7]:
# TODO
bikeStationPath = "201508_station_data.csv"

options = {
    'header': True,
    'inferSchema': True,
    'sep': ','
}

df = spark.read.options(**options).csv(bikeStationPath)

df.show(2)

+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 2 rows



In [8]:
df.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)



In [9]:
df.columns

['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'installation']

### 2. Lectura con Esquema

- Cargue archivo <code>.csv</code> con el esquema apropiado
- Generar `df` 
- Configure opciones faltantes
- Realice la exploración de datos del DF

In [13]:
ddl = 'station_id integer, name string, lat float, long float, dockcount integer, landmark string, installation string'

options = {
    'header': True,
    'sep': ','
}

df_schema = spark.read.options(**options).schema(ddl).csv(bikeStationPath)

df_schema.show(3)


+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...| 37.32973| -121.90178|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330696| -121.88898|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...| 37.33399|-121.894905|       11|San Jose|    8/6/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 3 rows



### 3. Escritura de DF

- Guarde el dataframe <code>productsDF</code>:
 - formato `parquet`
 - formato `csv`
 - formato `avro`

In [14]:
os.getcwd()

'/content'

In [15]:

path = '/content/bike_csv'

options = {
    'header': True,
    'sep': ','
}

df_schema.write.mode('overwrite').options(**options).csv(path)

In [16]:
os.listdir(path)

['._SUCCESS.crc',
 'part-00000-7112ae43-6ec6-4d7d-a65d-06cdc47d5fed-c000.csv',
 '.part-00000-7112ae43-6ec6-4d7d-a65d-06cdc47d5fed-c000.csv.crc',
 '_SUCCESS']

In [17]:
files.download(path + '/part-00000-7112ae43-6ec6-4d7d-a65d-06cdc47d5fed-c000.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###  ARTICULOS
-------------------------

- [Mostrar Primeras Filas del Df](https://sparkbyexamples.com/spark/show-top-n-rows-in-spark-pyspark/)